In [1]:
import sympy as sym
from sympy.abc import x, y, z
from sympy import Poly
from sympy import LT, LM, LC
from sympy import lcm

#WE CAN CHECK myGroebner WITH sym.groebner(f, x, y, z, order = "lex"), USE SUITABLE GENS AND ORDER

In [2]:
def myDiv(lst, f, t, gen, dom):
    a = [0] * len(lst)
    r = 0
    p = f
    while p != 0:
        i = 0
        divisionoccured = False
        while i<len(lst) and divisionoccured == False:
            if sym.div(Poly(LT(p, order = t), gen, domain = dom), Poly(LT(lst[i], order = t), gen, domain = dom))[1] == 0 :
                a[i] = a[i] + sym.div(Poly(LT(p, order = t), gen, domain = dom), Poly(LT(lst[i], order = t), gen, domain = dom))[0]
                p = p - sym.div(Poly(LT(p, order = t), gen, domain = dom), Poly(LT(lst[i], order = t), gen, domain = dom))[0] * lst[i]
                divisionoccured = True
            else:
                i = i + 1
        if divisionoccured == False:
            r = r + LT(p, order = t)
            p = p - LT(p, order = t)
    return a, r

In [3]:
def calS(f, g, t):
    x_gamma = lcm(LM(f, order = t), LM(g, order = t))
    S = x_gamma/LT(f, order = t) * f - x_gamma/LT(g, order = t) * g
    return S

In [4]:
def myGroebner(f, t, gen, dom):
    g = f.copy()
    rolling = True
    while rolling == True:
        g_control = g.copy()
        for i in range (len(g_control)-1):
            for j in range (i+1, len(g_control)):
                S = calS(g[i], g[j], t = t)
                rem = myDiv(g_control, S, t = t, gen = gen, dom = dom)[1]
                if rem !=0:
                    g.append(Poly(rem, gen, domain = dom))
        if g == g_control:
            rolling = False

    for i in range (len(g)):
        print(f"g{i} = {sym.latex(g[i].as_expr())}")
        
    return g

In [5]:
def myGroebnerCheck(g, t, gen, dom):
    dict_of_S={}
    for i in range(len(g) - 1):
        for j in range(i+1, len(g)):
            S = calS(g[i], g[j], t = t)      
            dict_of_S[i,j] = S

    for item in dict_of_S:
        print(f"S(g{item[0]}, g{item[1]}) = {sym.latex(dict_of_S[item].as_expr())}")
        print(f"Reminder of division with g = {myDiv(g, dict_of_S[item], t = t, gen = gen, dom = dom)[1]}")
        
    return

In [6]:
def myRedGroeb(f, t, gen, dom):
    #HAVING GROEBNER BASIS
    g = myGroebner(f, t = t, gen = gen, dom = dom)

    #HAVING MONIC GROEBNER BASIS
    for i in range(len(g)):
        g[i] = sym.monic(g[i])

    #MAKING LIST OF LT(g):
    lstLT = []
    for i in range (len(g)):
        lstLT.append(Poly(LT(g[i], order = t), gen, domain = dom))

    #CHANGING THE NON REDUCED BASIS INTO ZERO
    h = g.copy()
    for i in range (len(g)):
        while h[i] != 0 :
            denum = lstLT[0:i] + lstLT[i+1:]
            rem = myDiv(denum, Poly(LT(h[i], order = t), gen, domain = dom), t = t, gen = gen, dom = dom)[1]
            if rem == 0 :
                g[i] = 0
                break
            else:
                h[i] = h[i] - LT(h[i], order = t)

    #REMOVING THE ZERO BASIS
    g = list( filter( lambda a: a != 0, g ) )
    print(f"Reduced Groebner = {g}")
    return g

In [13]:
f0 = Poly(x*z - y, x, y, z, domain = 'QQ')
f1 = Poly(x*y + 2*z**2, x, y, z, domain = 'QQ')
f2 = Poly(y - z, x, y, z, domain = 'QQ')

f = [f0, f1, f2]
temp = calS(f0,f1,t="lex")
myDiv(f, temp, t="lex", gen=(x,y,z), dom="QQ")

([0, 0, Poly(-y - z, x, y, z, domain='QQ')], -2*z**3 - z**2)

In [17]:
f0 = Poly(x*z - y, x, y, z, domain = 'QQ')
f1 = Poly(x*y + 2*z**2, x, y, z, domain = 'QQ')
f2 = Poly(y - z, x, y, z, domain = 'QQ')
f3 = Poly(-2*z**3 - z**2, x, y, z, domain = "QQ")

f = [f0, f1, f2, f3]
temp = calS(f1,f2,t="lex")
myDiv(f, temp, t="lex", gen=(x,y,z), dom="QQ")

([Poly(1, x, y, z, domain='QQ'), 0, Poly(1, x, y, z, domain='QQ'), 0],
 2*z**2 + z)

In [ ]:
f0 = Poly(x*z - y, x, y, z, domain = 'QQ')
f1 = Poly(x*y + 2*z**2, x, y, z, domain = 'QQ')
f2 = Poly(y - z, x, y, z, domain = 'QQ')
f3 = Poly(-2*z**3 - z**2, x, y, z, domain = "QQ")
f4 = Poly(2*z**2 + z, x, y, z, domain = "QQ")

f = [f0, f1, f2, f3, f4]
temp = calS(f3,f4,t="lex")
myDiv(f, temp, t="lex", gen=(x,y,z), dom="QQ")

In [ ]:
#02

f0 = Poly(x*z - y, x, y, z, domain = 'QQ')
f1 = Poly(x*y + 2*z**2, x, y, z, domain = 'QQ')
f2 = Poly(y - z, x, y, z, domain = 'QQ')

f = [f0, f1, f2]
print(f)
print("---myGroebner---")
G = myGroebner(f, t = "lex", gen =(x,y,z), dom='QQ')